In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
import ast
import time
import math
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
def convertToTimestampArray(x):
    try:
        return [isoparse(d) for d in ast.literal_eval(x)]
    except:
        return np.nan

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
courseStudentsFilesPath = str(input())

In [ ]:
courseStudentsFiles = glob.glob(courseStudentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
courseStudentsData = pd.DataFrame()
for courseStudentsFile in courseStudentsFiles:
    courseStudentsData = courseStudentsData.append(pd.read_csv(courseStudentsFile), ignore_index = True)
    
courseStudentsData

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
courseStudentsData['metadata_event_time'] = courseStudentsData['metadata_event_time'].apply(parseTimestamp)
courseStudentsData.dropna(subset=['metadata_event_time'], inplace = True)
courseStudentsData = courseStudentsData.loc[(courseStudentsData['metadata_event_time'] >= startDate) & (courseStudentsData['metadata_event_time'] <= endDate)]
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseStudentsData.drop(['total'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
assignmentSubmissionsData = pd.read_csv(str(input()))
assignmentSubmissionsData

In [ ]:
assignmentSubmissionsData['metadata_event_time'] = assignmentSubmissionsData['metadata_event_time'].apply(convertToTimestampArray)
assignmentSubmissionsData = assignmentSubmissionsData.rename(columns={'metadata_event_time': 'Asub'})
assignmentSubmissionsData

In [ ]:
courseStudentsData = courseStudentsData.merge(assignmentSubmissionsData, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
assignmentAccessesData = pd.read_csv(str(input()))
assignmentAccessesData

In [ ]:
assignmentAccessesData['metadata_event_time'] = assignmentAccessesData['metadata_event_time'].apply(convertToTimestampArray)
assignmentAccessesData = assignmentAccessesData.rename(columns={'metadata_event_time': 'Aaccess'})
assignmentAccessesData

In [ ]:
courseStudentsData = courseStudentsData.merge(assignmentAccessesData, how = 'left', on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
#courseAssignmentDueDates = pd.read_csv(str(input()))
#courseAssignmentDueDates

In [ ]:
#courseAssignmentDueDates['body_due_at'] = courseAssignmentDueDates['body_due_at'].apply(isoparse)
#courseAssignmentDueDates = courseAssignmentDueDates.groupby(['metadata_context_id']).agg({'body_due_at': list}).reset_index()
#courseAssignmentDueDates

In [ ]:
#courseStudentsData = courseStudentsData.merge(courseAssignmentDueDates, how='left', on=['metadata_context_id'])
#courseStudentsData

In [ ]:
quizAccessesData = pd.read_csv(str(input()))
quizAccessesData

In [ ]:
quizAccessesData['metadata_event_time'] = quizAccessesData['metadata_event_time'].apply(convertToTimestampArray)
quizAccessesData = quizAccessesData.rename(columns={'metadata_event_time': 'Qaccess'})
quizAccessesData

In [ ]:
courseStudentsData = courseStudentsData.merge(quizAccessesData, how = 'left', on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
quizSubmissionData = pd.read_csv(str(input()))
quizSubmissionData

In [ ]:
quizSubmissionData['metadata_event_time'] = quizSubmissionData['metadata_event_time'].apply(convertToTimestampArray)
quizSubmissionData = quizSubmissionData.rename(columns={'metadata_event_time': 'Qsub', 'body_user_id': 'metadata_user_id'})
quizSubmissionData

In [ ]:
courseStudentsData = courseStudentsData.merge(quizSubmissionData, how = 'left', on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
discussionForumSubmissionGradedData = pd.read_csv(str(input()))
discussionForumSubmissionGradedData

In [ ]:
discussionForumSubmissionGradedData['metadata_event_time'] = discussionForumSubmissionGradedData['metadata_event_time'].apply(convertToTimestampArray)
discussionForumSubmissionGradedData = discussionForumSubmissionGradedData.rename(columns={'metadata_event_time': 'GDsub'})
discussionForumSubmissionGradedData

In [ ]:
courseStudentsData = courseStudentsData.merge(discussionForumSubmissionGradedData, how = 'left', on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
discussionForumAccessGradedData = pd.read_csv(str(input()))
discussionForumAccessGradedData

In [ ]:
discussionForumAccessGradedData['metadata_event_time'] = discussionForumAccessGradedData['metadata_event_time'].apply(convertToTimestampArray)
discussionForumAccessGradedData = discussionForumAccessGradedData.rename(columns={'metadata_event_time': 'GDaccess'})
discussionForumAccessGradedData

In [ ]:
courseStudentsData = courseStudentsData.merge(discussionForumAccessGradedData, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
def mergeAndSortTimestamps(df, columns):
    aggregatedTimestamps = []
    for column in columns:
        try:
            aggregatedTimestamps = aggregatedTimestamps + [float(math.floor((time.mktime(d.timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600))) for d in list(df[column])]
        except TypeError:
            aggregatedTimestamps = aggregatedTimestamps + []
    return sorted(aggregatedTimestamps)

In [ ]:
#courseStudentsData['A'] = courseStudentsData.apply(lambda df: mergeAndSortTimestamps(df, ['Asub', 'Aaccess', 'body_due_at', 'Qaccess', 'Qsub', 'GDsub', 'GDaccess']), axis = 1)
courseStudentsData['A'] = courseStudentsData.apply(lambda df: mergeAndSortTimestamps(df, ['Asub', 'Aaccess', 'Qaccess', 'Qsub', 'GDsub', 'GDaccess']), axis = 1)
courseStudentsData = courseStudentsData[['metadata_context_id', 'metadata_user_id', 'A']]
courseStudentsData

In [ ]:
courseStudentsData['hasA'] = courseStudentsData['A'].apply(lambda x: len(x) > 0)
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.loc[courseStudentsData['hasA'] == True]
courseStudentsData.drop(['hasA'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
discussionForumSubmissionUngradedData = pd.read_csv(str(input()))
discussionForumSubmissionUngradedData

In [ ]:
discussionForumSubmissionUngradedData['metadata_event_time'] = discussionForumSubmissionUngradedData['metadata_event_time'].apply(convertToTimestampArray)
discussionForumSubmissionUngradedData = discussionForumSubmissionUngradedData.rename(columns={'metadata_event_time': 'UDsub'})
discussionForumSubmissionUngradedData

In [ ]:
courseStudentsData = courseStudentsData.merge(discussionForumSubmissionUngradedData, how = 'left', on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
discussionForumAccessUngradedData = pd.read_csv(str(input()))
discussionForumAccessUngradedData

In [ ]:
discussionForumAccessUngradedData['metadata_event_time'] = discussionForumAccessUngradedData['metadata_event_time'].apply(convertToTimestampArray)
discussionForumAccessUngradedData = discussionForumAccessUngradedData.rename(columns={'metadata_event_time': 'UDaccess'})
discussionForumAccessUngradedData

In [ ]:
courseStudentsData = courseStudentsData.merge(discussionForumAccessUngradedData, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
commentsData = pd.read_csv(str(input()))
commentsData

In [ ]:
commentsData['metadata_event_time'] = commentsData['metadata_event_time'].apply(convertToTimestampArray)
commentsData = commentsData.rename(columns={'metadata_event_time': 'C'})
commentsData

In [ ]:
courseStudentsData = courseStudentsData.merge(commentsData, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
courseStudentsData['C'] = courseStudentsData.apply(lambda df: mergeAndSortTimestamps(df, ['UDsub', 'UDaccess', 'C']), axis = 1)
courseStudentsData = courseStudentsData[['metadata_context_id', 'metadata_user_id', 'A', 'C']]
courseStudentsData

In [ ]:
courseStudentsData['hasC'] = courseStudentsData['C'].apply(lambda x: len(x) > 0)
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.loc[courseStudentsData['hasC'] == True]
courseStudentsData.drop(['hasC'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
lessonAccesses = pd.read_csv(str(input()))
lessonAccesses

In [ ]:
lessonAccesses['metadata_event_time'] = lessonAccesses['metadata_event_time'].apply(convertToTimestampArray)
lessonAccesses

In [ ]:
lessonAccesses['metadata_event_time'] = lessonAccesses.apply(lambda df: mergeAndSortTimestamps(df, ['metadata_event_time']), axis = 1)
lessonAccesses = lessonAccesses.rename(columns={'metadata_event_time': 'L'})
lessonAccesses

In [ ]:
courseStudentsData = courseStudentsData.merge(lessonAccesses, on = ['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
#courseStudentsData = courseStudentsData.merge(courseAssignmentDueDates, on = ['metadata_context_id'])
#courseStudentsData

In [ ]:
#courseStudentsData = courseStudentsData.rename(columns = {'body_due_at': 'D'})
#courseStudentsData

In [ ]:
#courseStudentsData['D'] = courseStudentsData.apply(lambda df: mergeAndSortTimestamps(df, ['D']), axis = 1)
#courseStudentsData

In [ ]:
assignmentDelays = pd.read_csv(str(input()))
assignmentDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(assignmentDelays.rename(columns={"body_user_id": "metadata_user_id"}), how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
gradedDiscussionForumDelays = pd.read_csv(str(input()))
gradedDiscussionForumDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(gradedDiscussionForumDelays, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
quizzesDelays = pd.read_csv(str(input()))
quizzesDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(quizzesDelays, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
def averageDelays(df, columns):
    result = []
    for column in columns:
        if not math.isnan(df[column]):
            result.append(df[column])
    if(len(result) > 0):
        return float(np.mean(result))
    else:
        return np.nan

courseStudentsData['ave_delay_graded'] = courseStudentsData.apply(lambda df: averageDelays(df, ['ave_delay_' + dim for dim in ['A', 'GD', 'Q']]), axis = 1)
courseStudentsData.drop(['ave_delay_' + dim for dim in ['A', 'GD', 'Q']], axis = 1, inplace = True)
courseStudentsData

In [ ]:
ungradedDiscussionForumDelays = pd.read_csv(str(input()))
ungradedDiscussionForumDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(ungradedDiscussionForumDelays, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
commentsDelays = pd.read_csv(str(input()))
commentsDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(commentsDelays, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
courseStudentsData['ave_delay_ungraded'] = courseStudentsData.apply(lambda df: averageDelays(df, ['ave_delay_' + dim for dim in ['C', 'UD']]), axis = 1)
courseStudentsData.drop(['ave_delay_' + dim for dim in ['C', 'UD']], axis = 1, inplace = True)
courseStudentsData

In [ ]:
lessonsDelays = pd.read_csv(str(input()))
lessonsDelays

In [ ]:
courseStudentsData = courseStudentsData.merge(lessonsDelays, how = 'left', on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
courseStudentGrades = pd.read_csv(str(input()))
courseStudentGrades

In [ ]:
courseStudentGrades = courseStudentGrades.loc[courseStudentGrades['est_course_grade'] > 0]
courseStudentGrades

In [ ]:
courseStudentGrades['est_course_grade'] = courseStudentGrades['est_course_grade'].apply(lambda grade: float(math.floor(grade)))
courseStudentGrades

In [ ]:
courseStudentsData = courseStudentsData.merge(courseStudentGrades, on=['metadata_context_id', 'metadata_user_id'])
courseStudentsData

In [ ]:
courseModalities = pd.read_csv(str(input()))
courseModalities

In [ ]:
courseModalities = courseModalities.loc[~(courseModalities['Modality'] == '(~)')]
courseModalities['short_name'] = courseModalities['Subject Code'] + "-" + courseModalities['Section']
courseModalities.drop(['Subject Code', 'Section', 'Department', 'School'], axis = 1, inplace = True)
courseModalities

In [ ]:
courseModalities['Modality'] = courseModalities['Modality'].apply(lambda modality: modality if modality in ['(FULLY ONLINE)', '(FULLY ONSITE)'] else '(HYBRID)')
courseModalities

In [ ]:
courseList = pd.read_csv(str(input()))
courseList

In [ ]:
courseList = courseList[['canvas_course_id', 'short_name']]
courseList['canvas_course_id'] = courseList['canvas_course_id'].apply(lambda x: 'id_' + str(int(x) + 165820000000000000))
courseList

In [ ]:
courseModalities = courseList.merge(courseModalities, on=['short_name'])
courseModalities

In [ ]:
courseModalities = courseModalities.rename(columns={'canvas_course_id': 'metadata_context_id'})
courseModalities

In [ ]:
courseModalities = courseModalities.loc[~(courseModalities['Modality'] == '(FULLY ONSITE)')]
courseModalities

In [ ]:
courseModalities = courseModalities.loc[courseModalities['metadata_context_id'].isin(courseStudentsData['metadata_context_id'].unique())]
courseModalities

In [ ]:
courseModalities.groupby(['Modality'])['metadata_context_id'].count().to_frame('total_classes').reset_index()

In [ ]:
courseModalities['course_code'] = courseModalities['short_name'].apply(lambda x: x.split("-")[0])
courseModalities.drop(['short_name'], axis = 1, inplace = True)
courseModalities

In [ ]:
courseModalitiesCount = courseModalities.groupby(['course_code', 'Modality'])['metadata_context_id'].count().to_frame('total_classes').reset_index()
courseModalitiesCount

In [ ]:
courseModalitiesCount = courseModalitiesCount.pivot(columns = 'Modality', index='course_code', values='total_classes').reset_index()
courseModalitiesCount

In [ ]:
courseModalitiesCount = courseModalitiesCount.dropna()
courseModalitiesCount

In [ ]:
courseModalities = courseModalities.loc[courseModalities['course_code'].isin(courseModalitiesCount['course_code'].unique())]
courseModalities

In [ ]:
courseStudentsData = courseStudentsData.merge(courseModalities, on=['metadata_context_id'])
courseStudentsData

In [ ]:
courseStudentsData.to_csv("HawkesProcessesInput_Intercession_2022.csv", header = True, index = False)